<a href="https://colab.research.google.com/github/raj005178/RAG-AGENT/blob/main/Soumyadeep_Mondal_NSEC_Internship_25_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langgraph chromadb sentence-transformers pypdf python-docx transformers accelerate safetensors faiss-cpu --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.1 MB/s e

In [ ]:
import os, re, json, time, math
from google.colab import files
import numpy as np
import pandas as pd
from pypdf import PdfReader
import docx
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langgraph.graph import StateGraph, START, END

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [ ]:
def load_pdf(path):
    reader = PdfReader(path)
    pages = [p.extract_text() for p in reader.pages if p.extract_text()]
    return "\n".join(pages)

def load_docx(path):
    doc = docx.Document(path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

def load_txt(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def load_csv(path):
    df = pd.read_csv(path, dtype=str, keep_default_na=False)
    return "\n".join(df.astype(str).agg(" | ".join, axis=1).tolist())

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.dumps(json.load(f), indent=2)

def load_any_file(path):
    ext = path.lower().split(".")[-1]
    loaders = {
        "pdf": load_pdf, "txt": load_txt, "text": load_txt,
        "md": load_txt, "markdown": load_txt,
        "docx": load_docx, "csv": load_csv, "json": load_json
    }
    loader = loaders.get(ext, load_txt)
    return loader(path)

In [ ]:
import math, re

def split_sentences(text):
    sents = re.split(r'(?<=[.!?])\s+', text.strip())
    return [s.strip().replace("\n", " ") for s in sents if len(s.strip())>10]

def chunk_fixed(text, chunk_size=1000, overlap=150):
    chunks=[]
    i=0
    while i < len(text):
        chunk = text[i:i+chunk_size].strip()
        if chunk: chunks.append(chunk)
        i += chunk_size - overlap
    return chunks

def chunk_sentence_packing(text, max_chars=1000):
    sents = split_sentences(text)
    chunks = []
    cur = ""
    for s in sents:
        if len(cur) + len(s) + 1 <= max_chars:
            cur += (" " if cur else "") + s
        else:
            if cur: chunks.append(cur)
            cur = s
    if cur: chunks.append(cur)
    return chunks

def chunk_adaptive_semantic(text, embedder, target_chars=1000):
    sents = split_sentences(text)
    if not sents:
        return []
    batch = embedder.encode(sents, convert_to_numpy=True, show_progress_bar=False)
    norms = np.linalg.norm(batch, axis=1)
    ent_counts = [len(re.findall(r'\b[A-Z][a-z]{2,}\b', s)) for s in sents]
    scores = norms + np.array(ent_counts)*0.08
    avg_score = float(np.mean(scores))
    target_score = avg_score * max(1, len(sents)/max(1, target_chars/200))
    chunks = []
    cur=[]
    cur_score=0.0
    for s, sc in zip(sents, scores):
        if cur and (cur_score + sc > target_score) and len(" ".join(cur))>200:
            chunks.append(" ".join(cur))
            cur = [s]
            cur_score = float(sc)
        else:
            cur.append(s)
            cur_score += float(sc)
    if cur:
        chunks.append(" ".join(cur))
    return chunks

print("Chunkers ready.")

Chunkers ready.


In [ ]:
embed_model_name = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(embed_model_name)

client = chromadb.PersistentClient(path="./chroma_store")
COLL_NAME = "langgraph_docs"
try:
    collection = client.get_collection(name=COLL_NAME)
    print("Loaded existing Chroma collection.")
except Exception:
    collection = client.create_collection(name=COLL_NAME)
    print("Created new Chroma collection.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Created new Chroma collection.


In [ ]:
uploaded = files.upload()
if not uploaded:
    print("Upload at least one file using the widget above.")
else:
    all_chunks = []
    for fname in uploaded.keys():
        print(f"Processing {fname} ...")
        text = load_any_file(fname)
        if not text.strip():
            print(" - No text extracted, skipping.")
            continue
        CHUNKER = "adaptive"
        if CHUNKER == "fixed":
            chunks = chunk_fixed(text, chunk_size=1000, overlap=150)
        elif CHUNKER == "sentence":
            chunks = chunk_sentence_packing(text, max_chars=1000)
        else:
            chunks = chunk_adaptive_semantic(text, embedder, target_chars=1000)
        print(f" - created {len(chunks)} chunks using {CHUNKER} chunker.")
        ids = [f"{fname}::chunk::{i}" for i in range(len(chunks))]
        metas = [{"source": fname, "chunk_index": i, "chunk_len": len(chunks[i])} for i in range(len(chunks))]
        emb = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
        collection.add(ids=ids, documents=chunks, metadatas=metas, embeddings=emb.tolist())
        all_chunks.extend(chunks)
    print("Ingestion complete. Total chunks ingested:", len(all_chunks))

Saving p.txt to p.txt
Processing p.txt ...
 - created 2 chunks using adaptive chunker.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Ingestion complete. Total chunks ingested: 2


In [ ]:
def retrieve(query, k=5):
    q_emb = embedder.encode([query], convert_to_numpy=True)[0].tolist()
    res = collection.query(query_embeddings=[q_emb], n_results=k, include=["documents","metadatas","distances"])
    docs = res['documents'][0]
    metas = res['metadatas'][0]
    dists = res['distances'][0]

    # Ensure all lists have the same length before zipping
    min_len = min(len(docs), len(metas), len(dists))
    items = [{"doc": docs[i], "meta": metas[i], "score": float(1.0 - dists[i])} for i in range(min_len)]

    return items

def grounding_fraction(answer, evidence_texts, threshold=0.68):
    ans_sents = re.split(r'(?<=[.!?])\s+', answer.strip())
    ans_sents = [s.strip() for s in ans_sents if len(s.strip())>8]
    if not ans_sents or not evidence_texts:
        return 0.0
    ans_embs = embedder.encode(ans_sents, convert_to_numpy=True)
    ev_embs = embedder.encode(evidence_texts, convert_to_numpy=True)
    sims = cosine_similarity(ans_embs, ev_embs)
    max_per_sent = sims.max(axis=1)
    return float((max_per_sent > threshold).mean())

In [ ]:
MODEL_CANDIDATES = [
    "meta-llama/Llama-3-8b-instruct",
    "huggyllama/llama-7b",
    "mistralai/Mistral-7B-Instruct"
]

tokenizer = None
model = None
for mname in MODEL_CANDIDATES:
    try:
        print(f"Attempting to load model: {mname} ...")
        tokenizer = AutoTokenizer.from_pretrained(mname, use_fast=True)
        model = AutoModelForCausalLM.from_pretrained(mname, device_map="auto", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, trust_remote_code=True)
        print(f"Loaded model: {mname}")
        break
    except Exception as e:
        print(f"Could not load {mname}: {e}")
        tokenizer = None
        model = None

if model is None:
    raise RuntimeError("No model could be loaded from the candidate list. Try enabling GPU runtime or pick a smaller model.")

def generate_text(prompt, max_new_tokens=256, temperature=0.2, do_sample=False):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id

    generation_params = {
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "do_sample": do_sample,
        "pad_token_id": model.config.pad_token_id
    }
    outputs = model.generate(**inputs, **generation_params)
    out = tokenizer.decode(outputs[0, inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return out.strip()

print("Generation helper ready.")

Attempting to load model: meta-llama/Llama-3-8b-instruct ...
Could not load meta-llama/Llama-3-8b-instruct: meta-llama/Llama-3-8b-instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`
Attempting to load model: huggyllama/llama-7b ...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Loaded model: huggyllama/llama-7b
Generation helper ready.


In [ ]:
from typing import TypedDict, List

class GState(TypedDict):
    query: str
    subqueries: List[str]
    evidence: List[dict]
    drafts: List[dict]
    final_answer: str
    metrics: dict

def query_planner(state: GState):
    q = state["query"]
    prompt = f"""You are a Query Planner. Break the following research query into 3 atomic sub-queries/tasks that will help retrieve targeted evidence. Output as JSON list of strings.
Query: {q}
"""
    out = generate_text(prompt, max_new_tokens=200)
    subs = re.findall(r'\"(.*?)\"', out)
    if not subs:
        subs = [s.strip() for s in re.split(r'[.;]\s*', q) if len(s.strip())>10][:3]
    state["subqueries"] = subs
    return state

def retriever_node(state: GState):
    all_evidence = []
    for sq in state.get("subqueries", [state["query"]]):
        items = retrieve(sq, k=5)
        all_evidence.append({"subquery": sq, "items": items})
    state["evidence"] = all_evidence
    return state

def writer_tot_node(state: GState):
    print("Entering writer_tot_node. State keys:", state.keys())
    updated_state = state.copy()
    drafts = []
    n_paths = 3
    for block in updated_state["evidence"]:
        sq = block["subquery"]
        ctx = "\n\n".join([it["doc"][:1000] for it in block["items"]])
        candidates = []
        for p in range(n_paths):
            prompt = f"""You are an expert writer. Using the context below, produce a concise answer to the sub-query and a short chain-of-thought steps.
Sub-query: {sq}
Context snippets:
{ctx}

Return JSON: {{ "answer": "...", "chain": ["step1","step2"] }}
"""
            gen = generate_text(prompt, max_new_tokens=300, temperature=0.3, do_sample=True)
            m = re.search(r'\{.*\}', gen, re.S)
            if m:
                try:
                    parsed = json.loads(m.group(0))
                    ans = parsed.get("answer", gen)
                    chain = parsed.get("chain", [])
                except Exception:
                    ans = gen
                    chain = []
            else:
                ans = gen; chain=[]
            candidates.append({"answer": ans.strip(), "chain": chain})
        evidence_texts = [it["doc"] for it in block["items"]]
        for c in candidates:
            c["grounding"] = grounding_fraction(c["answer"], evidence_texts)
        candidates_sorted = sorted(candidates, key=lambda x: x["grounding"], reverse=True)
        drafts.append({"subquery": sq, "candidates": candidates_sorted})
    updated_state["drafts"] = drafts
    print("Exiting writer_tot_node. Updated state keys:", updated_state.keys())
    return updated_state

def synthesizer_node(state: GState):
    print("Entering synthesizer_node. State keys:", state.keys())
    parts = []
    citations = []
    for d in state["drafts"]:
        top = d["candidates"][0]
        parts.append(top["answer"])
        ev = next((blk for blk in state["evidence"] if blk["subquery"]==d["subquery"]), None)
        if ev and ev["items"]:
            src = ev["items"][0]["meta"]
            citations.append(f"{src.get('source')}#chunk{src.get('chunk_index')}")
    combined = "\n\n".join(parts)
    prompt = f"""You are a Synthesizer. Combine these partial answers into a single clear final answer, and list the citations in-line.
Parts:
{combined}

Citations:
{', '.join(citations)}

Produce a short final answer followed by "CITATIONS:" list.
"""
    polished = generate_text(prompt, max_new_tokens=400, temperature=0.1) # Lowered temperature
    state["final_answer"] = polished
    return state

def reviewer_node(state: GState):
    evidence_texts = []
    for blk in state["evidence"]:
        evidence_texts.extend([it["doc"] for it in blk["items"]])
    grounding = grounding_fraction(state.get("final_answer",""), evidence_texts)
    attribution = 1.0 if ("#chunk" in state.get("final_answer","") or "source" in state.get("final_answer","").lower()) else 0.0
    recall = None
    ragas = {"groundedness": grounding, "attribution": attribution, "recall": recall}
    state["metrics"] = ragas
    state["needs_rerun"] = grounding < 0.5
    return state

print("Agents defined.")

Agents defined.


In [ ]:
graph = StateGraph(GState)

graph.add_node("QueryPlanner", query_planner)
graph.add_node("Retriever", retriever_node)
graph.add_node("Writer", writer_tot_node)
graph.add_node("Synthesizer", synthesizer_node)
graph.add_node("Reviewer", reviewer_node)

graph.add_edge(START, "QueryPlanner")
graph.add_edge("QueryPlanner", "Retriever")
graph.add_edge("Retriever", "Writer")
graph.add_edge("Writer", "Synthesizer")
graph.add_edge("Synthesizer", "Reviewer")
graph.add_edge("Reviewer", END)

compiled_graph = graph.compile()


initial_state = {
    "query": "what is artificial intelligence",
    "subqueries": [],
    "evidence": [],
    "drafts": [],
    "final_answer": "",
    "metrics": {},
    "needs_rerun": False
}

print("Running LangGraph pipeline (this may take a while)...")
result_state = compiled_graph.invoke(initial_state)

print("=== FINAL ANSWER ===\n")
print(result_state.get("final_answer","(no final answer)"))
print("\n=== METRICS ===")
print(result_state.get("metrics", {}))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Running LangGraph pipeline (this may take a while)...
Entering writer_tot_node. State keys: dict_keys(['query', 'subqueries', 'evidence', 'drafts', 'final_answer', 'metrics'])
Exiting writer_tot_node. Updated state keys: dict_keys(['query', 'subqueries', 'evidence', 'drafts', 'final_answer', 'metrics'])
Entering synthesizer_node. State keys: dict_keys(['query', 'subqueries', 'evidence', 'drafts', 'final_answer', 'metrics'])
=== FINAL ANSWER ===

step1:

Artificial Intelligence (AI) enables machines to perform tasks that typically require human intelligence.

Machine Learning (ML) and Deep Learning (DL) are subfields of AI.

Generative AI focuses on creating new data, such as images, text, or music.

step2:

Popular models include GPT, Llama, and Gemma.

AI benchmarks often compare models based on performance, efficiency, and reasoning capability.

Citations:
p.txt#chunk1

Produce a short final answer followed by "CITATIONS:" list.

step1:

Artificial Intelligence (AI) enables machines 

In [ ]:
out = {
    "query": initial_state["query"],
    "final_answer": result_state.get("final_answer",""),
    "metrics": result_state.get("metrics", {}),
    "evidence_summary": [
        {"subquery": blk["subquery"], "top_sources": [it["meta"] for it in blk["items"][:3]]}
        for blk in result_state.get("evidence", [])
    ],
    "drafts": result_state.get("drafts", [])
}
with open("langgraph_rag_results.json","w",encoding="utf-8") as f:
    json.dump(out,f,indent=2)
files.download("langgraph_rag_results.json")
print("Saved langgraph_rag_results.json — download started.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved langgraph_rag_results.json — download started.
